In [1]:
import P_Cyg as pc
import numpy as np
import pandas as pd
from glob import glob
from astropy.time import Time

In [2]:
find_detector = lambda files : [f[4:7]    for f in files]
find_date = lambda files : [f[14:18]+'-'+f[18:20]+'-'+f[20:22]    for f in files]
sig = lambda x : float("{0:.3f}".format(x))

# H$\alpha$

### Observed

In [3]:
datadir = 'C:/Users/Keyan/Desktop/Science/Data/P-Cygni/Data/'
rtxt = glob(datadir + 'hpolccd*r_hw.fits.txt')
btxt = glob(datadir + 'hpolccd*b_hw.fits.txt')
rettxt = glob(datadir + '*ret*.txt')
ccdtxt = []
for rf in rtxt:
    for bf in btxt:
        if rf[63:71]==bf[63:71]:
            ccdtxt.append([rf,bf])
ccddfs = [[pd.read_csv(f[0],delim_whitespace=True,header=0,names=["Wavelength","Flux","Q","U","Error"]),pd.read_csv(f[1],delim_whitespace=True,header=0,names=["Wavelength","Flux","Q","U","Error"])]    for f in ccdtxt]
ccddfs = [pd.concat([p[0],p[1][p[1].Wavelength < 6100]]).sort_values("Wavelength")    for p in ccddfs]
retdfs = [pd.read_csv(f,delim_whitespace=True,header=0,names=["Wavelength","Flux","Q","U","Error"])    for f in rettxt]
obdfs = retdfs + ccddfs

In [4]:
hmask = lambda df: df[(df.Wavelength>6540) & (df.Wavelength<6600)]
ohdfs = [hmask(df)    for df in obdfs]

In [5]:
ohq,ohu = [sig(df.Q.mean())    for df in ohdfs],[sig(df.U.mean())    for df in ohdfs]

### Intrinsic

In [6]:
intrinsic_dir = "C:/Users/Keyan/Desktop/Science/Data/P-Cygni/Data/intrinsic_pols/"
files = glob(intrinsic_dir+"*")
retfiles = [f    for f in files if f[72]=='o']
rfiles = [f    for f in files if f[72]=='r']
bfiles = [f    for f in files if f[72]=='b']
matched = []
for rf in rfiles:
    for bf in bfiles:
        if rf[64:69]==bf[64:69]:
            matched.append([rf,bf])
matched = np.array(matched)
mdfs = [[pd.read_csv(f[0]),pd.read_csv(f[1])]    for f in matched]
mdfs = [pd.concat([p[0],p[1][p[1].Wavelength < 6100]]).sort_values("Wavelength")    for p in mdfs]
rdfs = [pd.read_csv(f)    for f in files if f[72]=='r']
bdfs = [pd.read_csv(f)    for f in files if f[72]=='b']
retdfs = [pd.read_csv(f)     for f in retfiles]
alldfs = retdfs+mdfs
t = [float(f[64:69])    for f in retfiles] + [float(f[0][64:69])    for f in matched]
t = [round(x)    for x in t]

In [7]:
ihdfs = [hmask(df)    for df in alldfs]

In [8]:
ihq,ihu = [sig(df.Q.mean())    for df in ihdfs],[sig(df.U.mean())    for df in ihdfs]
he = [sig(np.sqrt(np.sum(df.Error**2))/df.Error.size)    for df in ihdfs]

## Pfew

In [9]:
pq,pu,ppol,ppos=np.array(pc.pfew(alldfs))

### H table

In [10]:
htable = pd.DataFrame({"MJD":t,
                       "oq":ohq,
                       "ou":ohu,
                      "iq":ihq,
                      "iu":ihu,
                      "Error":he,
                      "pq":pq,
                      "pu":pu},index=None)
htable = htable.drop([27,64]).reset_index()

# $V$-Band

### Observed

In [11]:
ccd_dir = 'C:/Users/Keyan/Desktop/Science/Data/P-Cygni/Data/synthetic_filters/VBand_CCD_WithErrors.txt'
ret_dir = 'C:/Users/Keyan/Desktop/Science/Data/P-Cygni/Data/synthetic_filters/VBand_Reticon_WithErrors.txt'

In [12]:
ccd = pd.read_csv(ccd_dir,delim_whitespace=True)
ret = pd.read_csv(ret_dir,delim_whitespace=True)
vtable = pd.concat([ret,ccd]).reset_index().drop(columns='index')
vtable["Detector"] = find_detector(vtable["File/Night"])
vtable["Date"] = find_date(vtable["File/Night"])
vtable["MJD"] = Time(list(vtable.Date.values),format='fits',location=(43,-89,0)).mjd
vtable["Error"] = np.sqrt(vtable.Internal_Error **2 + vtable.Systematic_Error **2)

In [13]:
vtable["Error"] = [sig(x)    for x in vtable["Error"]]
vtable.q = [sig(x)    for x in vtable.q]
vtable.u = [sig(x)    for x in vtable.u]


### Intrinsic

### Vtable

In [14]:
vtable = pd.DataFrame({"Detector":vtable.Detector,
                      "MJD":vtable.MJD,
                      "q":vtable.q,
                      "u":vtable.u,
                      "Error":vtable.Error},index=None)

# Big Table

In [16]:
dformat = lambda dlist: ["Reticon" if d == 'ret' else "CCD"    for d in dlist]
bigtable = pd.DataFrame({"Detector":dformat(vtable.Detector),
                        "MJD":htable.MJD,
                        "ovq":vtable.q,
                        "ovu":vtable.u,
                        "ove":vtable.Error,
                        "ohq":htable.oq,
                        "ohu":htable.ou,
                        "ihq":htable.iq,
                        "ihu":htable.iu,
                        "pq":htable.pq,
                        "pu":htable.pu,
                        "he":htable.Error,}, index=None)
bigtable.to_csv("C:/Users/Keyan/Desktop/Science/Data/P-Cygni/Data/bigtable.txt")
print(bigtable.to_latex(index=False))

\begin{tabular}{lrrrrrrrrrrr}
\toprule
Detector &    MJD &    ovq &    ovu &    ove &    ohq &    ohu &    ihq &    ihu &        pq &        pu &     he \\
\midrule
 Reticon &  47668 &  0.305 &  0.997 &  0.020 &  0.356 &  0.966 & -0.045 & -0.036 &  0.031145 &  0.039727 &  0.042 \\
 Reticon &  47672 &  0.611 &  1.057 &  0.020 &  0.447 &  1.091 &  0.072 &  0.097 & -0.049555 &  0.095136 &  0.029 \\
 Reticon &  47697 &  0.289 &  1.128 &  0.020 &  0.246 &  1.006 & -0.114 &  0.007 & -0.086243 & -0.033151 &  0.029 \\
 Reticon &  47714 &  0.437 &  0.863 &  0.021 &  0.300 &  0.940 & -0.106 & -0.050 & -0.067948 & -0.025028 &  0.037 \\
 Reticon &  47808 &  0.527 &  1.106 &  0.020 &  0.377 &  0.991 & -0.024 & -0.017 & -0.062784 & -0.031090 &  0.029 \\
 Reticon &  47822 &  0.174 &  0.826 &  0.020 &  0.297 &  0.828 & -0.094 & -0.181 &  0.009155 & -0.063080 &  0.019 \\
 Reticon &  47913 &  0.716 &  1.317 &  0.021 &  0.452 &  0.987 &  0.080 &  0.012 & -0.027512 & -0.107352 &  0.048 \\
 Reticon &  4791